<a href="https://colab.research.google.com/github/AnnMokhova/nlp_2020/blob/main/hw2/hw2_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 2. Извлечение коллокаций + NER

'Beauty.txt.gz' взято из http://snap.stanford.edu/data/web-Amazon-links.html

In [1]:
from google.colab import drive
drive.mount('gdrive')

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/My Drive/

/content/gdrive/My Drive


In [3]:
#pip install simplejson

In [4]:
import gzip
import simplejson
import re
import pandas as pd

In [5]:
def parse(filename):
    f = gzip.open(filename, 'rt')
    entry = {}
    for l in f:
        l = l.strip()
        colonPos = l.find(':')
        if colonPos == -1:
            yield entry
            entry = {}
            continue
        eName = l[:colonPos]
        rest = l[colonPos+2:]
        entry[eName] = rest
        yield entry


data = []

for e in parse('Beauty.txt.gz'):
    line = simplejson.dumps(e)
    if 'review/text' in line:
        line = re.findall('"review/text": ".*?"', line)
        data.append(line)

In [6]:
data = [re.sub('"review/text": ', '', ''.join(line)) for line in data]

In [7]:
# убираем повторяющиеся отзывы
data = list(set(data))

In [8]:
import nltk

In [9]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [10]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [11]:
from nltk.stem import WordNetLemmatizer

In [12]:
lemmatizer = WordNetLemmatizer()

def lemmatize_eng(text):
    text = text.lower()
    text = re.sub('[.|,|"|`]', '', text)
    tokens = [w for w in nltk.word_tokenize(text) if w not in '1234567890,.!?-();:""«»—_–#' and '.' not in w]
    lemmas = [lemmatizer.lemmatize(t) for t in tokens]
    return ' '.join(lemmas)

In [13]:
df = pd.DataFrame({'text': data})
df['normalized_text'] = df['text'].apply(lemmatize_eng)
df.head()

,text,normalized_text
0,"""I gave my son a sample of the Kiss My Face Or...",i gave my son a sample of the kiss my face org...
1,"""I was shocked to see how well this product wo...",i wa shocked to see how well this product work...
2,"""Works better than I expected, was able to cov...",work better than i expected wa able to cover t...
3,"""This smells so good! If you like warm scents ...",this smell so good if you like warm scent like...
4,"""I disagree with the other reviewer. It's true...",i disagree with the other reviewer it 's true ...


## Способы поиска упоминаний товаров в отзывах

1. Ruled-based подход: составление шаблонов n-грамм по типу "*тип косметического продукта* + n-1 слов после типа товара", поиск соотвутствующих n-грамм, выделение из них названия товаров (удаление первого слова в шаблоне). Для реализации этого способа необходим примерный перечень наиболее популярных средств, которые представлены в отзывах. Минус: возможное отсутствие готовых метаданных о содержании отзывов; тогда необходимо либо использовать очень большой список шаблонов, либо перед составлением шаблонов сделать частотный словарь существительных в текстах отзывов, и по ним определить вручную главные слова для шаблонов. Плюс: если использовать разное количество слов в n-граммах, можно получить список, из которого с большой вероятностью можно выделить нужные названия. Однако невсегда в текстах люди используют формат тип продукта + название, а просто пишут название, поэтому шаблоны такого вида могут не выявить упоминание товара + такой способ больше подходит для поиска названий торговых марок, а настоящая задача заключается в поиске упоминаний типов товаров
2. Условные случайные поля: использование готовой библиотеки stanza. Плюсы: находит сложные зависимости; готовый инструмент. Минус: будет находить вообще все именованные сущности в отзывах, а не только упоминания товаров. Для избавления от этого недостатка можно выбирать только сущности определенного типа, которые указывали бы на продукт (например, организации). Другой недостаток: не будет находить такие упоминания товаров как "телефон", "модель" и т.д.
3. Составление словаря ключевых слов + расширение с помощью эмбеддингов + поиск н-грамм по списку ключевых слов. Плюс: в данных текстах, скорее всего, мало семантической неоднозначности, тексты одной тематики. Минусы: не учитываются названия брендов, если не использовать дополнительно более длинные n-граммы.

## Составление словаря ключевых слов и расширение с помощью эмбеддингов

In [14]:
!wget http://vectors.nlpl.eu/repository/20/5.zip

--2020-11-23 13:47:25--  http://vectors.nlpl.eu/repository/20/5.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 574653290 (548M) [application/zip]
Saving to: ‘5.zip.2’

5.zip.2             100%[===================>] 548.03M  23.3MB/s    in 32s     

2020-11-23 13:47:57 (17.3 MB/s) - ‘5.zip.2’ saved [574653290/574653290]



In [15]:
!unzip '5.zip'

Archive:  5.zip
replace meta.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: meta.json               
replace model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: model.bin               
replace model.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: model.txt               
replace README? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: README                  


In [16]:
from gensim.models.keyedvectors import KeyedVectors

model = KeyedVectors.load_word2vec_format('model.bin', binary=True)
model.save_word2vec_format('model.txt', binary=False)

In [17]:
def word2vec_eng(word, model=model):
    if word in model:
        return model.most_similar(word, topn=1)[0][0]

In [18]:
#pip install python-rake

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
import RAKE
from nltk.corpus import stopwords

In [21]:
stop = stopwords.words('english')

Выберем ключевые слова в каждом отзыве с помощью RAKE, составим топ ключевых слов всего корпуса и выделим наиболее релевантные существительные

In [22]:
rake = RAKE.Rake(stop)

In [23]:
all_kw = []
for i, row in df.iterrows():
    kw = rake.run(df['normalized_text'][i], maxWords=3, minFrequency=1)
    for k in kw:
        all_kw.append(k[0])

In [24]:
from collections import Counter
Counter(all_kw).most_common(150)

[('product', 47038),
 ('n', 38785),
 ('use', 34436),
 ('love', 27675),
 ('hair', 26306),
 ('used', 23852),
 ('wa', 23479),
 ('using', 18905),
 ('get', 17869),
 ('one', 17609),
 ('like', 17172),
 ('bought', 16273),
 ('skin', 15659),
 ('make', 13873),
 ('time', 13692),
 ('year', 13620),
 ('find', 13545),
 ('day', 12706),
 ('work', 12705),
 ('well', 11863),
 ('ha', 11552),
 ('doe n', 11284),
 ('smell', 11155),
 ('buy', 10533),
 ('found', 10406),
 ('tried', 9820),
 ('doe', 9819),
 ('think', 9728),
 ('try', 9571),
 ('great', 9519),
 ('much', 9413),
 ('good', 9024),
 ('would', 8962),
 ('price', 8939),
 ('face', 8908),
 ('lot', 8826),
 ('know', 8823),
 ('amazon', 8748),
 ('got', 8519),
 ('dry', 8300),
 ('need', 8288),
 ('give', 8175),
 ('put', 8174),
 ('want', 7886),
 ('say', 7877),
 ('scent', 7831),
 ('ca n', 7723),
 ('go', 7607),
 ('see', 7285),
 ('week', 7271),
 ('last', 7118),
 ('purchased', 6884),
 ('way', 6880),
 ('keep', 6374),
 ('happy', 6203),
 ('thought', 6083),
 ('take', 5946),
 ('

In [86]:
entities =  ['product', 'hair', 'skin', 'face', 'parfume', 'fragrance', 'shampoo', 
             'lotion', 'item', 'shower', 'conditioner', 'cream']
result = []

# расширение списка дескрипторов с помощью эмбеддингов
for e in entities:
    new_e = word2vec_eng(e)
    if new_e not in entities and new_e != None:
        result.append(new_e)
entities.extend(result)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [87]:
entities

['product',
 'hair',
 'skin',
 'face',
 'parfume',
 'fragrance',
 'shampoo',
 'lotion',
 'item',
 'shower',
 'conditioner',
 'cream',
 'consumer',
 'beard',
 'mucous',
 'overcome',
 'perfume',
 'trinket',
 'bathroom',
 'dehumidifier',
 'chocolate']

## Биграммы с полученными сущностями

In [88]:
def left_context(text, entities=entities):
    text = text.split()
    res = []
    for w in text:
        if w in entities:
            n = text.index(w)
            try:
                res.append(' '.join([text[n-1], text[n]]))
            except IndexError:
                continue
    return res

In [89]:
def right_context(text, entities=entities):
    text = text.split()
    res = []
    for w in text:
        if w in entities:
            n = text.index(w)
            try:
                res.append(' '.join([text[n], text[n+1]]))
            except IndexError:
                continue
    return res

In [90]:
n_gramms = []
for i, row in df.iterrows():
    n_gramms.extend(left_context(df['normalized_text'][i]))
    n_gramms.extend(right_context(df['normalized_text'][i]))

In [91]:
words = ' '.join(n_gramms)
count_dict_words = Counter(words.split())
count_dict_bigram = Counter(n_gramms)

In [92]:
# размер всего корпуса
N = len(''.join(df['normalized_text']).split())

In [93]:
N

13407765

# Ранжирование биграмм по коллокационным метрикам

In [94]:
from nltk.metrics import *

In [95]:
bam = BigramAssocMeasures

In [96]:
# список выделенных сочетаний без повторений 
ngr_list = list(set(n_gramms))

In [97]:
student_t = {}
chi_sq = {}
pmi = {}
for n in ngr_list:
    student_t[n] = bam.student_t(count_dict_bigram[n], (count_dict_words[n.split()[0]], 
                                                        count_dict_words[n.split()[1]]), N)
    try:
        chi_sq[n] = bam.chi_sq(count_dict_bigram[n], (count_dict_words[n.split()[0]], 
                                                          count_dict_words[n.split()[1]]), N)
    except ZeroDivisionError:
        chi_sq[n] = 0
    pmi[n] = bam.pmi(count_dict_bigram[n], (count_dict_words[n.split()[0]], 
                                                        count_dict_words[n.split()[1]]), N)

### t-score

In [98]:
student_t = list(student_t.items())
student_t.sort(key=lambda i: i[1], reverse=True)
student_t[:100]

[('this product', 241.39499677195764),
 ('my hair', 198.8117157936411),
 ('my skin', 140.41074757153976),
 ('my face', 108.87295970316607),
 ('hair and', 108.83724046558943),
 ('the product', 106.83496723018041),
 ('product i', 106.37673838250775),
 ('product for', 101.5123704347583),
 ('product is', 89.36878982506649),
 ('your hair', 85.12382779660255),
 ('hair i', 83.81009060488375),
 ('skin and', 83.52698010335799),
 ('hair dryer', 82.32412811285086),
 ('product and', 78.003098200727),
 ('hair is', 73.51689361248167),
 ('your skin', 73.11440463210077),
 ('this perfume', 70.70719199654063),
 ('curly hair', 70.30457946807134),
 ('sensitive skin', 69.6724083306759),
 ('great product', 69.50552984885773),
 ('dry skin', 64.72956635100938),
 ('hair that', 64.62432390431321),
 ('the skin', 64.3968960516785),
 ('product it', 62.68648307380003),
 ('product that', 62.42537805328627),
 ('skin i', 61.52297676723383),
 ('this item', 60.638089588010445),
 ('this shampoo', 58.89969784586411),
 ('h

### хи-квадрат

In [99]:
chi_sq = list(chi_sq.items())
chi_sq.sort(key=lambda i: i[1], reverse=True)
chi_sq[:100]

[('this product', 2096874.6537405218),
 ('mucous membrane', 1915393.28571403),
 ('tacky trinket', 1532313.371428531),
 ('my hair', 1112709.101151513),
 ('consumer report', 1112465.224376499),
 ('my skin', 548704.9295787879),
 ('my face', 531209.944400166),
 ('sensitive skin', 514951.2339483805),
 ('eye cream', 410898.44103360345),
 ('shower gel', 375278.0263907055),
 ('hair dryer', 366840.32928688795),
 ('face wash', 336668.1292690976),
 ('nasal mucous', 319230.90476294),
 ('body lotion', 316555.72561237606),
 ('de parfume', 310669.3085143854),
 ('product for', 296364.19996057806),
 ('dry skin', 287582.69262206246),
 ('hand cream', 286210.71752970567),
 ('night cream', 285738.4472078389),
 ('curly hair', 265286.0256938251),
 ('overcome genetics', 223461.76666659332),
 ('overcome thatthanks', 223461.76666659332),
 ('product i', 215280.6618061443),
 ('trinket laundry', 212819.9206374612),
 ('bathroom sink', 194149.77110185605),
 ('your skin', 193642.4070934743),
 ('leave-in conditioner',

###PMI

In [100]:
pmi = list(pmi.items())
pmi.sort(key=lambda i: i[1], reverse=True)
pmi[:100]

[('mucous membrane', 19.86921051018318),
 ('tacky trinket', 19.547282415295815),
 ('nasal mucous', 18.284248009462026),
 ('overcome genetics', 17.769674836632266),
 ('overcome thatthanks', 17.769674836632266),
 ('trinket laundry', 17.699285508740868),
 ('watery mucous', 17.284248009462026),
 ('ugly mucous', 16.54728241529582),
 ('tiny trinket', 16.284248009462026),
 ('tryingbest parfume', 15.208959882157789),
 ('picking parfume', 15.208959882157789),
 ('favorate parfume', 15.208959882157789),
 ('overcome addiction', 15.18471233591111),
 ('consumer rep', 14.22741678686535),
 ('frugal consumer', 14.22741678686535),
 ('consumer familiarize', 14.22741678686535),
 ('consumer fraud', 14.22741678686535),
 ('consumer advocate', 14.22741678686535),
 ('fad consumer', 14.22741678686535),
 ('consumer consequently', 14.22741678686535),
 ('health-conscious consumer', 14.22741678686535),
 ('haira consumer', 14.22741678686535),
 ('cologne/perfume consumer', 14.22741678686535),
 ('choosy consumer', 14.

Метрика хи-квадрат показывает наилучший результат, так как она ранжирует выше коллокации, где содержатся значимые по смыслу слова (меньше артиклей или метоимений по сравнению с результатами метрики t-score) и при этом выбирает наиболее обобщенные словосочетания (по сравнению с результатами метрики pmi)

### Группировка сочетаний по NE

In [101]:
best_collocations = [col[0] for col in chi_sq[:100]]

In [102]:
examples = {}

for col in best_collocations:
    if col.split()[0] in entities:
        if col.split()[0] in examples.keys():
            examples[col.split()[0]].append(col)
        else:
            examples[col.split()[0]] = [col]
    else:
        if col.split()[1] in examples.keys():
            examples[col.split()[1]].append(col)
        else:
            examples[col.split()[1]] = [col]

In [103]:
for e in entities[:5]:
    print('NE:', e)
    print('\n'.join(examples[e]))
    print('.....')

NE: product
this product
product for
product i
the product
great product
product is
good product
product that
product to
product work
product it
product wa
product and
product because
other product
.....
NE: hair
my hair
hair dryer
curly hair
hair and
your hair
fine hair
thick hair
long hair
wavy hair
hair i
hair that
hair is
hair color
frizzy hair
thin hair
straight hair
.....
NE: skin
my skin
sensitive skin
dry skin
your skin
oily skin
skin and
skin tone
skin feel
combination skin
skin care
fair skin
prone skin
the skin
dead skin
skin i
skin feeling
.....
NE: face
my face
face wash
your face
entire face
.....
NE: parfume
de parfume
.....


### Способы группировки по синонимам:
Частеречная разметка послеследующих и предшествующих NE слов (предположительно, перед существительном следует прилагательное, а после -- глагол). Составить список из слов, которые входят в левый и правый контекст, объединить синонимы по векторной близости слов одной части речи